In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
tf.set_random_seed(777)  # reproducibility
nb_classes = 10
keep_prob = tf.placeholder(tf.float32)
X = tf.placeholder(tf.float32,shape=(None,784))
Y = tf.placeholder(tf.float32,shape=(None,nb_classes))

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# layer1
with tf.name_scope("layer1") as scope:
    W1 = tf.get_variable("W1",shape=(784,512),initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([512]),name="bias1")
    L1 = tf.nn.relu(tf.matmul(X,W1)+b1)
    L1 = tf.nn.dropout(L1,keep_prob=keep_prob)
    
    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", L1)

# layer2
with tf.name_scope("layer2") as scope:
    W2 = tf.get_variable("W2",shape=(512,512),initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([512]),name="bias2")
    L2 = tf.nn.relu(tf.matmul(L1,W2)+b2)
    L2 = tf.nn.dropout(L2,keep_prob=keep_prob)
    
    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    layer2_hist = tf.summary.histogram("layer2", L2)
    
# layer3
with tf.name_scope("layer3") as scope:
    W3 = tf.get_variable("W3",shape=(512,512),initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([512]),name="bias3")
    L3 = tf.nn.relu(tf.matmul(L2,W3)+b3)
    L3 = tf.nn.dropout(L3,keep_prob=keep_prob)
    
    w3_hist = tf.summary.histogram("weights3", W3)
    b3_hist = tf.summary.histogram("biases3", b3)
    layer3_hist = tf.summary.histogram("layer3", L3)

# layer4
with tf.name_scope("layer4") as scope:
    W4 = tf.get_variable("W4",shape=(512,512),initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([512]),name="bias4")
    L4 = tf.nn.relu(tf.matmul(L3,W4)+b4)
    L4 = tf.nn.dropout(L4,keep_prob=keep_prob)
    
    w4_hist = tf.summary.histogram("weights4", W4)
    b4_hist = tf.summary.histogram("biases4", b4)
    layer4_hist = tf.summary.histogram("layer4", L4)

# output layer
with tf.name_scope("layer5") as scope:
    W5 = tf.get_variable("W5",shape=(512,nb_classes),initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([nb_classes]),name="bias5")
    hypothesis = tf.matmul(L4,W5) +b5
    
    w5_hist = tf.summary.histogram("weights5", W5)
    b5_hist = tf.summary.histogram("biases5", b5)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

#cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis),axis=1))
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y)
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(cost_i)
    cost_scalar = tf.summary.scalar("cost",cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
predicted = tf.arg_max(hypothesis,dimension=1)
is_correct= tf.equal(predicted,tf.arg_max(Y,dimension=1))
accuracy = tf.reduce_mean(tf.cast(is_correct,dtype=tf.float32))
accuracy_scalar = tf.summary.scalar("accuracy",accuracy)

with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/EXAMPLE")
    writer.add_graph(sess.graph) # Show the graph
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size) # 한 epoch에 몇번의 batch를 실행해야 하는지
        for step in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val,summary,_ = sess.run([cost,merged_summary,train],feed_dict={
                X : batch_xs,
                Y : batch_ys,
                keep_prob : 0.7
            })
            writer.add_summary(summary, global_step=step)
            avg_cost += cost_val / total_batch
        print("Epoch",epoch+1,"Cost : ",avg_cost)
    
    accuracy = sess.run(accuracy,feed_dict={
        X : mnist.test.images,
        Y : mnist.test.labels,
        keep_prob : 1.0
    })
    print("Accuracy : ",accuracy)
        

Epoch 1 Cost :  0.485057080137
Epoch 2 Cost :  0.17492504724
Epoch 3 Cost :  0.133461480879
Epoch 4 Cost :  0.112356460718
Epoch 5 Cost :  0.0933711894949
Epoch 6 Cost :  0.0852105632915
Epoch 7 Cost :  0.0762664886475
Epoch 8 Cost :  0.067546607935
Epoch 9 Cost :  0.0645035094332
Epoch 10 Cost :  0.0584281745794
Epoch 11 Cost :  0.0562146977568
Epoch 12 Cost :  0.0533668008605
Epoch 13 Cost :  0.0500391882224
Epoch 14 Cost :  0.0502267064656
Epoch 15 Cost :  0.0471721950952
Accuracy :  0.9813
